In [ ]:
import arcpy


db = arcpy.env.workspace = './data/tdm_conflation.gdb'
arcpy.Describe(db)

In [ ]:
sr = arcpy.SpatialReference(2223)

def project(in_data, target_sr, out_name):


    in_sr = arcpy.Describe(in_data).spatialReference
    tr = arcpy.ListTransformations (in_sr, target_sr)
    datum_conversion = ''
    if(len(tr)>0):
        datum_conversion = arcpy.ListTransformations (in_sr, target_sr)[0]

    arcpy.Project_management(
        in_dataset = in_data, 
        out_dataset = out_name,
        out_coor_system = target_sr,
        transform_method = datum_conversion)

In [ ]:
future_rd_raw = 'SunCloudFutureOnlyLinksVolumesCapacityV3'
out_name = 'unbuilt_future_roads '
sr = arcpy.SpatialReference(2223)
project(future_rd_raw,sr, out_name)


In [ ]:
target = out_name

arcpy.management.CalculateField(
    in_table="unbuild_future_roads",
    field="volume_2050",
    expression="max(!ABdayVOL50!,!BAdayVOL50!,!TOTdayVOL50!)",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

arcpy.management.CalculateField(
    in_table="unbuild_future_roads",
    field="capacity_2050",
    expression="!ABCAPACITY2050!+!BACAPACITY2050!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
fc = 'unbuilt_future_roads'
target_fields = [
    ('suncloud_id', 'sun_cloud_id', 'Segment ID'),
    ('volume_2050','volume_2050','Daily Volume 2050'),
    ('model', 'source', 'Source'),
    ('st_name', 'route_id', 'Route ID'),
    ('capacity_2050','capacity_2050', 'Capacity 2050'),
    ('vc_2050','vc_2050','Volume/Capacity Ratio 2050')
]
for current, new, alias in target_fields:
    arcpy.management.AlterField(fc, current, new, alias)

In [12]:
req_fields = ['sun_cloud_id','capacity_2050', 'volume_2050','source','route_id', 'vc_2050']

all_fields = arcpy.ListFields(fc)
f_names = [f.name for f in all_fields if f.required == False]

to_delete = [i for i in f_names if i not in req_fields]
to_delete

['ID',
 'DIR',
 'LENGTH',
 'ABLANES2050',
 'BALANES2050',
 'ABCAPACITY2050',
 'BACAPACITY2050',
 'ABdayVOL50',
 'BAdayVOL50',
 'TOTdayVOL50',
 'Comments']

In [15]:
arcpy.DeleteField_management(fc, to_delete)

<Result './data/tdm_conflation.gdb\\unbuilt_future_roads'>